In [1]:
import requests
import pandas as pd
import datetime

In [2]:
from pathlib import Path

In [3]:
#Daten laden mit Path, damit die Namen nachher sauber geschrieben sind und ich danach filtern kann
df_raw = pd.read_csv(Path('./SVP_Vorstoesse_14_23_mit_Resultat_bereinigt.csv'), encoding='UTF-8')

In [4]:
df_raw['Datum'] = pd.to_datetime(df_raw['Datum'])

In [5]:
df = df_raw.copy()

# Komischen Leerschlag ersetzen: regex muss auf "True" sein, sonst wird es nur ersetzt, wenn der ganze Inhalt der Zelle dem Suchwert entspricht
df = df.replace(u"\xa0", " ", regex=True)


In [6]:
#Test: Alle Vorstösse von Samuel Balsiger
df[(df["Erstunterzeichner"] == "Samuel Balsiger") | (df["Mitunterzeichner"] == "Samuel Balsiger")]

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus,Entscheid,_entscheidbereinigt
19,Zuweisung von Notwohnungen an Familien mit unt...,Schriftliche Anfrage,ee74826c671c41b0bc482d9181b738ad,2014-10-22,Samuel Balsiger,SVP,Peter Schick,SVP,Abgeschlossen,NaN,NaN
23,"Beratungs- und Anlaufstellen in der Stadt, Umf...",Schriftliche Anfrage,5004e78db5da4aea8651bf69bcc4743f,2014-11-19,Samuel Balsiger,SVP,Roberto Bertozzi,SVP,Abgeschlossen,NaN,NaN
27,"Verein Effekta, Richtlinien zur Zusammenarbeit...",Schriftliche Anfrage,3092e7f99a684beea9e5ca229102a6a7,2014-11-26,Samuel Balsiger,SVP,Roberto Bertozzi,SVP,Abgeschlossen,NaN,NaN
31,Einschätzung der Sicherheitslage der jüdischen...,Schriftliche Anfrage,8f7acff9a71849beae9b24672b767e50,2015-01-21,Samuel Balsiger,SVP,Roland Scheck,SVP,Abgeschlossen,NaN,NaN
32,"Kunst im öffentlichen Raum, Hintergründe zum g...",Schriftliche Anfrage,3b7e665068db4cdca2bfdbd298b86ce6,2015-01-21,Roland Scheck,SVP,Samuel Balsiger,SVP,Abgeschlossen,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
634,Warteliste der Stiftung Alterswohnungen der St...,Schriftliche Anfrage,19bc4fc68956466b957bd17928a9ae53,2023-09-20,Samuel Balsiger,SVP,Walter Anken,SVP,Abgeschlossen,NaN,NaN
635,Stiftung Alterswohnungen der Stadt Zürich (SAW...,Postulat,34edb617af2647be91142bb573da2c0c,2023-09-20,Samuel Balsiger,SVP,Walter Anken,SVP,InBearbeitung,NaN,NaN
636,Dauerhafte Nutzung des Gesundheitszentrums für...,Postulat,e53e84ea75dd43a5bb7832d3f9a248f7,2023-09-20,Samuel Balsiger,SVP,Walter Anken,SVP,InBearbeitung,NaN,NaN
645,Bewilligungen für Foodtrucks auch für öffentli...,Postulat,c7078c2b00784c699e4b32b090063074,2023-11-08,Samuel Balsiger,SVP,Sebastian Zopfi,SVP,InBearbeitung,NaN,NaN


In [7]:
#Anzahl Vorstösse pro Politiker, im ersten Schritt werden die beiden Spalten zusammengenommen, damit ich beide durchsuchen kann
politiker = pd.concat([df['Erstunterzeichner'], df['Mitunterzeichner']])
politiker_ausgezählt = politiker.value_counts()

In [8]:
politiker_ausgezählt

Stephan Iten         184
Samuel Balsiger      155
Martin Götzl         134
Stefan Urech          71
Derek Richter         67
                    ... 
Martina Zürcher        1
Serap Kahriman         1
Matthias Probst        1
David Ondraschek       1
Anthony Goldstein      1
Name: count, Length: 86, dtype: int64

## Wie häufig geht die SVP kooperationen ein und mit wem?

In [9]:
#Gibt alle Vorstösse heraus, die SVPler gemeinsam mit anderen Parteimitgliedern eingereicht haben
df_koop = df[(df["Partei Erstunterzeichner"] != "SVP") | (df["Partei Mitunterzeichner"] != "SVP")]
df_koop

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus,Entscheid,_entscheidbereinigt
17,Schliessung der Postfachanlagen in Zürich Ries...,Interpellation,06e9c070d7f74ce99666cc20b004a298,2014-10-22,Nina Fehr Düsel,SVP,Karin Weyermann,Die Mitte,Abgeschlossen,NaN,NaN
45,"Umwandlung von Parkverboten in Halteverbote, H...",Schriftliche Anfrage,5578fb502af9449c820aea36ad568898,2015-03-25,Markus Hungerbühler,CVP,Urs Fehr,SVP,Abgeschlossen,NaN,NaN
71,Sammlung von Bioabfällen und Produktion von Bi...,Interpellation,4b96be5fb5564d33b6e5ffc20cccff2c,2015-11-25,Urs Fehr,SVP,Mario Mariani,CVP,Abgeschlossen,NaN,NaN
78,"Angabe der Nationalität bei Polizeimeldungen, ...",Schriftliche Anfrage,e9946040325f49b38ae7282d496aec23,2016-01-13,Martin Götzl,SVP,NaN,NaN,Abgeschlossen,NaN,NaN
93,Auswirkungen der geplanten Tempo 30-Zonen an d...,Schriftliche Anfrage,06ba9a1d470c47db8bdc9d3e579b6c88,2016-03-23,Stefan Urech,SVP,Elisabeth Schoch,FDP,Abgeschlossen,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
616,Beschränkung des Anstiegs der Lohnkosten der S...,Postulat,d4d76a34d7e54405aaeb04f72ac7fd56,2023-04-19,Martin Götzl,SVP,Isabel Garcia,FDP,Abgeschlossen,Ablehnung,Ablehnung
617,Erhalt der Tennisinfrastruktur beim Neubau des...,Schriftliche Anfrage,3e6dc240c5544349a2c2cac28b683232,2023-05-24,Martin Götzl,SVP,Patrick Tscherrig,SP,Abgeschlossen,NaN,NaN
641,Weisungen zum Wohnungsbau und Baurechtsvergabe...,Postulat,1854b57582a441ebbe568a5fd68ca834,2023-11-01,Martin Götzl,SVP,Anthony Goldstein,FDP,InBearbeitung,NaN,NaN
642,"Umzonung eines Grünraums, inkl. Weg und Bach, ...",Motion,818758a69c2b4200940599d10dc9609d,2023-11-01,Matthias Probst,Grüne,Stephan Iten,SVP,InBearbeitung,NaN,NaN


In [10]:
#Hier noch die Gesamtzahl der SVP-Vorstösse, um sie ins Verhältnis zu den Kooperationen zu setzen
len(df)

654

In [11]:
#SVPler mit den häufigsten Kooperationen: dazu value_counts auf die df_koop angewandt
#Interessant ist am Ergebnis auch, dass niemand so viele Vorstösse mit SVPlern einreicht wie SP-Politiker Urs Helfenstein
politiker = pd.concat([df_koop['Erstunterzeichner'], df_koop['Mitunterzeichner']])
politiker_ausgezählt = politiker.value_counts()
politiker_ausgezählt.head(20)

Stefan Urech           24
Stephan Iten           18
Martin Götzl           14
Urs Helfenstein         8
Elisabeth Schoch        7
Christian Huser         6
Susanne Brunner         6
Andreas Egli            6
Roger Bartholdi         5
Reto Brüesch            5
Balz Bürgisser          5
Yasmine Bourgeois       4
Ernst Danner            3
Urs Fehr                3
Frank Rühli             2
Mischa Schiwow          2
Pascal Lamprecht        2
Walter Anken            2
Sabine Koch             2
Alan David Sangines     2
Name: count, dtype: int64

In [12]:
#Fraktionschef Samuel Balsiger hat nie einen Vorstoss mit Mitgliedern anderer Parteien eingereicht
df_koop[(df_koop["Erstunterzeichner"] == "Samuel Balsiger") | (df_koop["Mitunterzeichner"] == "Samuel Balsiger")]

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus,Entscheid,_entscheidbereinigt


In [13]:
#Hier ist aufgelistet, mit welchen Parteien die SVP häufig kooperiert
parteien = pd.concat([df_koop['Partei Erstunterzeichner'], df_koop['Partei Mitunterzeichner']])
parteien_ausgezählt = parteien.value_counts()
parteien_ausgezählt.head(20)

SVP          90
FDP          43
SP           19
Grüne         8
GLP           6
CVP           4
AL            3
Die Mitte     3
EVP           3
Name: count, dtype: int64

**Interpretation der Ergebnisse**
Die SVP geht insgesamt selten Kooperationen ein, was klar dafür spricht, dass sie im Rat sehr isoliert ist. Fast die Hälfte der Kooperationen macht sie mit der SVP, die anderen Parteien kommen selten vor. Andere Parteien sind da viel aktiver. Ich vergleiche die SVP dazu mit einer bürgerlichen Partei (FDP), einer Partei aus der Mitte (GLP) und einer linken Partei (SP)

## Vergleich mit anderen Parteien

**Zuerst die FDP**

In [14]:
#Daten laden mit Path, damit die Namen nachher sauber geschrieben sind und ich danach filtern kann
df_raw = pd.read_csv(Path('./FDP_Vorstoesse_14_23_mit_Resultat_bereinigt.csv'), encoding='UTF-8')

In [15]:
df_raw['Datum'] = pd.to_datetime(df_raw['Datum'])

In [16]:
df_fdp = df_raw.copy()

# Komischen Leerschlag ersetzen: regex muss auf "True" sein, sonst wird es nur ersetzt, wenn der ganze Inhalt der Zelle dem Suchwert entspricht
df_fdp = df_fdp.replace(u"\xa0", " ", regex=True)


In [17]:
len(df_fdp)

517

In [18]:
df_koop_fdp = df_fdp[(df_fdp["Partei Erstunterzeichner"] != "FDP") | (df_fdp["Partei Mitunterzeichner"] != "FDP")]
df_koop_fdp

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus,Entscheid,_entscheidbereinigt
2,"Restaurant Belvoir-Park, Verzicht auf den Bau ...",Postulat,96fe23887b7545299ff0562836335eef,2014-01-22,Michael Baumer,FDP,NaN,NaN,InBearbeitung,"Überweisung, Frist 24 Monate",Überweisung
4,"Teilrevision der Bau- und Zonenordnung (BZO), ...",Schriftliche Anfrage,e67c768f1c224330b68b363599ccb9be,2014-02-05,Marco Denoth,SP,Heinz F. Steger,FDP,Abgeschlossen,NaN,NaN
6,Vergabepraxis günstiger Wohnungen durch die St...,Schriftliche Anfrage,f87e14210ae1446db6773537291748aa,2014-03-12,Albert Leiser,FDP,Markus Hungerbühler,CVP,Abgeschlossen,NaN,NaN
7,Einführung einer Jugendinitiative als Instrume...,Postulat,4940b23f724d4d6cb4a21aed8f043f63,2014-03-19,Isabel Garcia,FDP,Matthias Wiesmann,GLP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
8,"Sechseläutenplatz, Nutzung für politische Vera...",Schriftliche Anfrage,ceb42fa107044b5d8d26143420d8daeb,2014-03-26,Urs Egger,FDP,NaN,NaN,Abgeschlossen,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
497,"Einsatz von Microsoft Office 365, Anzahl Tenan...",Schriftliche Anfrage,2540e4419ed44e17b450033573ed43ac,2023-08-30,Flurin Capaul,FDP,Urs Riklin,Grüne,Abgeschlossen,NaN,NaN
502,Attraktivität und Lebendigkeit der Zürcher Inn...,Interpellation,2f526026287745dc91932a1d934e0daf,2023-09-13,Pascal Lamprecht,SP,Përparim Avdili,FDP,InBearbeitung,NaN,NaN
504,Preisstruktur der städtischen Kindertagesstätt...,Postulat,0ee00994db24493e85f6692a345a5263,2023-10-04,Mélissa Dufournet,FDP,David Ondraschek,Die Mitte,InBearbeitung,NaN,NaN
505,Lockerung der Bewilligungspraxis für Gastrobet...,Postulat,21355d5ae78e4f5cbe4bb875d8190ce0,2023-10-04,Severin Meier,SP,Martin Bürki,FDP,InBearbeitung,NaN,NaN


In [19]:
#Hier ist aufgelistet, mit welchen Parteien die FDP häufig kooperiert
parteien = pd.concat([df_koop_fdp['Partei Erstunterzeichner'], df_koop_fdp['Partei Mitunterzeichner']])
parteien_ausgezählt_fdp = parteien.value_counts()
parteien_ausgezählt_fdp.head(20)

FDP          243
SP            71
GLP           62
SVP           43
AL            15
Grüne         15
CVP           13
EVP            8
Die Mitte      8
Parteilos      2
Name: count, dtype: int64

**Interpretation der Ergebnisse** Die FDP geht viel häufiger Kooperationen ein. Fast die Hälfte ihrer Vorstösse reicht sie mit anderen Parteien ein. Interessant ist auch, dass sie nicht mit ihrem bürgerlichen Partner der SVP am häufigsten zusammenarbeitet, sondern mit SP und GLP. Offensichtlich ist die FDP weit weniger isoliert als die SVP, die eigentlich nur bei der anderen bürgerlichen Partei, der FDP, Anschluss findet.

**Wie sieht es bei der GLP aus?**

In [20]:
#Daten laden mit Path, damit die Namen nachher sauber geschrieben sind und ich danach filtern kann
df_raw = pd.read_csv(Path('./GLP_Vorstoesse_14_23_mit_Resultat_bereinigt.csv'), encoding='UTF-8')

In [21]:
df_raw['Datum'] = pd.to_datetime(df_raw['Datum'])

In [22]:
df_glp = df_raw.copy()

# Komischen Leerschlag ersetzen: regex muss auf "True" sein, sonst wird es nur ersetzt, wenn der ganze Inhalt der Zelle dem Suchwert entspricht
df_glp = df_glp.replace(u"\xa0", " ", regex=True)

In [23]:
len(df_glp)

329

In [24]:
df_koop_glp = df_glp[(df_glp["Partei Erstunterzeichner"] != "GLP") | (df_glp["Partei Mitunterzeichner"] != "GLP")]
df_koop_glp

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus,Entscheid,_entscheidbereinigt
0,"Juliastrasse, Einrichtung einer Begegnungszone...",Postulat,1ae08ec9dfc0465ba1c5491de2584218,2014-02-05,Jean-Daniel Strub,SP,Martin Luchsinger,GLP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
1,Kommunale Wohnüberbauung auf dem Tramdepot Har...,Postulat,a5573cb80d054660a8565a27eab1a841,2014-02-26,Martin Luchsinger,GLP,Jean-Claude Virchaux,CVP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
2,Durchgehende Veloverbindung aus dem Lettenquar...,Postulat,94f2be0b2acd4eab8e6d7c743304b799,2014-03-05,Simone Brander,SP,Guido Trevisan,GLP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
3,Studie zur Wirkung der Mobilitätskampagne des ...,Schriftliche Anfrage,ef16480862ec4387991f9b736a9378ba,2014-03-12,Samuel Dubno,GLP,Walter Angst,AL,Abgeschlossen,NaN,NaN
4,Einführung einer Jugendinitiative als Instrume...,Postulat,4940b23f724d4d6cb4a21aed8f043f63,2014-03-19,Isabel Garcia,FDP,Matthias Wiesmann,GLP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
...,...,...,...,...,...,...,...,...,...,...,...
320,"Nutzung der Kasernenwiese, Öffnung für kommerz...",Schriftliche Anfrage,986d08fad0e44f7891ddfd0b4d429ae1,2023-09-06,Marco Denoth,SP,Beat Oberholzer,GLP,Abgeschlossen,NaN,NaN
321,Baugesuch der Stadt Zürich an der Förrlibuckst...,Schriftliche Anfrage,1f3f68958c3a4ee18834eafc05fe3061,2023-09-13,Sven Sobernheim,GLP,NaN,NaN,Abgeschlossen,NaN,NaN
322,Zusätzliche grossflächige Wandmalereien an stä...,Postulat,d1f64d45598241dba03ce3235f669480,2023-09-27,Severin Meier,SP,Guy Krayenbühl,GLP,InBearbeitung,NaN,NaN
324,"Versorgungsengpässe bei Arzneimitteln, betroff...",Schriftliche Anfrage,5ab193cebc1a48559449633b072e7c49,2023-11-22,Matthias Renggli,SP,Patrick Hässig,GLP,InBearbeitung,NaN,NaN


In [25]:
#Hier ist aufgelistet, mit welchen Parteien die GLP häufig kooperiert
parteien = pd.concat([df_koop_glp['Partei Erstunterzeichner'], df_koop_glp['Partei Mitunterzeichner']])
parteien_ausgezählt_glp = parteien.value_counts()
parteien_ausgezählt_glp.head(20)

GLP          218
SP            84
FDP           62
Grüne         42
AL            13
SVP            6
Die Mitte      3
EVP            3
CVP            1
Name: count, dtype: int64

**Interpretation der Ergebnisse**: Die GLP reicht fast zwei Drittel ihrer Vorstösse mit anderen Parteien ein. Der Unterscheid zur SVP fällt damit noch mehr auf. Die GLP setzt bei der Zusammenarbeit vor allem auf linke Parteien, arbeitet aber immerhin am zweithäufigsten mit der FDP zusammen.

**Wie sieht es bei der SP aus?**

In [26]:
#Daten laden mit Path, damit die Namen nachher sauber geschrieben sind und ich danach filtern kann
df_raw = pd.read_csv(Path('./SP_Vorstoesse_14_23_mit_Resultat_bereinigt.csv'), encoding='UTF-8')

In [27]:
df_raw['Datum'] = pd.to_datetime(df_raw['Datum'])

In [28]:
df_sp = df_raw.copy()

# Komischen Leerschlag ersetzen: regex muss auf "True" sein, sonst wird es nur ersetzt, wenn der ganze Inhalt der Zelle dem Suchwert entspricht
df_sp = df_sp.replace(u"\xa0", " ", regex=True)

In [29]:
len(df_sp)

832

In [30]:
df_koop_sp = df_sp[(df_sp["Partei Erstunterzeichner"] != "SP") | (df_sp["Partei Mitunterzeichner"] != "SP")]
df_koop_sp

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus,Entscheid,_entscheidbereinigt
0,Stellenwert und Bedeutung der Quartierzeitunge...,Schriftliche Anfrage,59510f153e2843a7a18ba5559fb148c2,2014-01-08,Michael Kraft,SP,NaN,NaN,Abgeschlossen,NaN,NaN
1,Umsetzung der Bewilligungspflicht des «Hooliga...,Schriftliche Anfrage,10fcdfe3849243868504a94906c451a2,2014-01-15,Matthias Probst,Grüne,Florian Utz,SP,Abgeschlossen,NaN,NaN
2,Verstösse der Vertragspartnerin für die Strass...,Schriftliche Anfrage,bdfe511a27a241ebabe4a7dc69d4d8f4,2014-01-22,Walter Angst,AL,Kyriakos Papageorgiou,SP,Abgeschlossen,NaN,NaN
5,"Juliastrasse, Einrichtung einer Begegnungszone...",Postulat,1ae08ec9dfc0465ba1c5491de2584218,2014-02-05,Jean-Daniel Strub,SP,Martin Luchsinger,GLP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
6,"Teilrevision der Bau- und Zonenordnung (BZO), ...",Schriftliche Anfrage,e67c768f1c224330b68b363599ccb9be,2014-02-05,Marco Denoth,SP,Heinz F. Steger,FDP,Abgeschlossen,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
827,"Hagenholzstrasse, Einführung von Tempo 30 und ...",Postulat,aa6584e0ea454f80829a8fcfdfcb454b,2023-11-29,Heidi Egger,SP,Matthias Probst,Grüne,InBearbeitung,NaN,NaN
828,Einsatz von Doppelgelenk-Trolleybussen bei sch...,Schriftliche Anfrage,f80960ad45a74f1e80e5a1cf2e3c16d7,2023-12-06,Ann-Catherine Nabholz,GLP,Ivo Bieri,SP,InBearbeitung,NaN,NaN
829,Umgestaltung des Sozialinspektorats hinsichtli...,Postulat,16285bcb96c84790bb7723bd44c7c98c,2023-12-06,Luca Maggi,Grüne,Anjushka Früh,SP,InBearbeitung,"Überweisung, Frist 24 Monate",Überweisung
830,Verstärkte Berücksichtigung von Mehrfachnutzun...,Postulat,657741aac79440cda18dc77082591f78,2023-12-06,Pascal Lamprecht,SP,Tanja Maag Sturzenegger,AL,InBearbeitung,NaN,NaN


In [31]:
#Hier ist aufgelistet, mit welchen Parteien die SP häufig kooperiert
parteien = pd.concat([df_koop_sp['Partei Erstunterzeichner'], df_koop_sp['Partei Mitunterzeichner']])
parteien_ausgezählt_sp = parteien.value_counts()
parteien_ausgezählt_sp.head(20)

SP           544
Grüne        209
GLP           84
FDP           71
AL            62
SVP           19
Die Mitte     17
CVP            6
EVP            5
Name: count, dtype: int64

**Interpretation der Ergebnisse**: Auch die SP kooperiert sehr häufig mit anderen Parteien, wie auch die GLP reicht sie fast zwei Drittel ihrer Vorstösse mit anderen Parteien ein. Wenig überraschend tut sie das am häufigsten mit ihrer "Schwesterpartei", den Grünen. Aber auch mit GLP und FDP arbeitet sie häufig zusammen.

## Resultate speichern

In [32]:
#DataFrame erstellen
df_zusammenarbeit_sp = pd.DataFrame(parteien_ausgezählt_sp)
df_zusammenarbeit_sp

,count
SP,544
Grüne,209
GLP,84
FDP,71
AL,62
SVP,19
Die Mitte,17
CVP,6
EVP,5


In [33]:
#Die eigene Partei rauslöschen, die logischerweise bei jeder Kooperation dabei ist
df_zusammenarbeit_sp.drop("SP", inplace=True)

In [34]:
df_zusammenarbeit_sp

,count
Grüne,209
GLP,84
FDP,71
AL,62
SVP,19
Die Mitte,17
CVP,6
EVP,5


In [35]:
#Die Spalten von Mitte und CVP addieren
df_zusammenarbeit_sp.loc['Die Mitte', 'count'] += df_zusammenarbeit_sp.loc['CVP', 'count']

# Neue Spalte "Mitte/CVP" benennen
df_zusammenarbeit_sp.rename(index={'Die Mitte': 'Mitte/CVP'}, inplace=True)

# Die alte CVP-Spalte löschen
df_zusammenarbeit_sp.drop('CVP', inplace=True)

df_zusammenarbeit_sp

,count
Grüne,209
GLP,84
FDP,71
AL,62
SVP,19
Mitte/CVP,23
EVP,5


In [36]:
df_sp_partner = df_zusammenarbeit_sp.sort_values(by='count', ascending=False)

In [37]:
df_sp_partner

,count
Grüne,209
GLP,84
FDP,71
AL,62
Mitte/CVP,23
SVP,19
EVP,5


In [38]:
df_sp_partner = df_sp_partner.rename_axis('Kooperationsparteien_SP')
df_sp_partner.reset_index(inplace=True)
df_sp_partner

,Kooperationsparteien_SP,count
0,Grüne,209
1,GLP,84
2,FDP,71
3,AL,62
4,Mitte/CVP,23
5,SVP,19
6,EVP,5


In [39]:
#Diese Resultate speichern
df_sp_partner.to_csv('Kooperationen_SP.csv', index=False)

**nun für die SVP**

In [40]:
#DataFrame erstellen
df_zusammenarbeit_svp = pd.DataFrame(parteien_ausgezählt)
df_zusammenarbeit_svp

,count
SVP,90
FDP,43
SP,19
Grüne,8
GLP,6
CVP,4
AL,3
Die Mitte,3
EVP,3


In [41]:
#Die eigene Partei rauslöschen, die logischerweise bei jeder Kooperation dabei ist
df_zusammenarbeit_svp.drop("SVP", inplace=True)

In [42]:
#Die Spalten von Mitte und CVP addieren
df_zusammenarbeit_svp.loc['Die Mitte', 'count'] += df_zusammenarbeit_svp.loc['CVP', 'count']

# Neue Spalte "Mitte/CVP" benennen
df_zusammenarbeit_svp.rename(index={'Die Mitte': 'Mitte/CVP'}, inplace=True)

# Die alte CVP-Spalte löschen
df_zusammenarbeit_svp.drop('CVP', inplace=True)

df_zusammenarbeit_svp = df_zusammenarbeit_svp.rename_axis('Kooperationsparteien_SVP')

df_zusammenarbeit_svp.reset_index(inplace=True)
df_zusammenarbeit_svp

,Kooperationsparteien_SVP,count
0,FDP,43
1,SP,19
2,Grüne,8
3,GLP,6
4,AL,3
5,Mitte/CVP,7
6,EVP,3


In [43]:
df_zusammenarbeit_svp.to_csv('Kooperationen_SVP.csv', index=False)

**nun für die FDP**

In [44]:
#DataFrame erstellen
df_zusammenarbeit_fdp = pd.DataFrame(parteien_ausgezählt_fdp)
df_zusammenarbeit_fdp

,count
FDP,243
SP,71
GLP,62
SVP,43
AL,15
Grüne,15
CVP,13
EVP,8
Die Mitte,8
Parteilos,2


In [45]:
#Die eigene Partei rauslöschen, die logischerweise bei jeder Kooperation dabei ist
df_zusammenarbeit_fdp.drop("FDP", inplace=True)

#Die Spalten von Mitte und CVP addieren
df_zusammenarbeit_fdp.loc['Die Mitte', 'count'] += df_zusammenarbeit_fdp.loc['CVP', 'count']

# Neue Spalte "Mitte/CVP" benennen
df_zusammenarbeit_fdp.rename(index={'Die Mitte': 'Mitte/CVP'}, inplace=True)

# Die alte CVP-Spalte löschen
df_zusammenarbeit_fdp.drop('CVP', inplace=True)

df_zusammenarbeit_fdp = df_zusammenarbeit_fdp.rename_axis('Kooperationsparteien_FDP')

df_zusammenarbeit_fdp.reset_index(inplace=True)
df_zusammenarbeit_fdp

,Kooperationsparteien_FDP,count
0,SP,71
1,GLP,62
2,SVP,43
3,AL,15
4,Grüne,15
5,EVP,8
6,Mitte/CVP,21
7,Parteilos,2


In [46]:
df_zusammenarbeit_fdp.to_csv('Kooperationen_FDP.csv', index=False)

**nun für die GLP**

In [47]:
#DataFrame erstellen
df_zusammenarbeit_glp = pd.DataFrame(parteien_ausgezählt_glp)
df_zusammenarbeit_glp

,count
GLP,218
SP,84
FDP,62
Grüne,42
AL,13
SVP,6
Die Mitte,3
EVP,3
CVP,1


In [48]:
#Die eigene Partei rauslöschen, die logischerweise bei jeder Kooperation dabei ist
df_zusammenarbeit_glp.drop("GLP", inplace=True)

#Die Spalten von Mitte und CVP addieren
df_zusammenarbeit_glp.loc['Die Mitte', 'count'] += df_zusammenarbeit_glp.loc['CVP', 'count']

# Neue Spalte "Mitte/CVP" benennen
df_zusammenarbeit_glp.rename(index={'Die Mitte': 'Mitte/CVP'}, inplace=True)

# Die alte CVP-Spalte löschen
df_zusammenarbeit_glp.drop('CVP', inplace=True)

df_zusammenarbeit_glp = df_zusammenarbeit_glp.rename_axis('Kooperationsparteien_GLP')

df_zusammenarbeit_glp.reset_index(inplace=True)

df_zusammenarbeit_glp

,Kooperationsparteien_GLP,count
0,SP,84
1,FDP,62
2,Grüne,42
3,AL,13
4,SVP,6
5,Mitte/CVP,4
6,EVP,3


In [49]:
df_zusammenarbeit_glp.to_csv('Kooperationen_GLP.csv', index=False)